In [32]:
import scipy.io
import numpy as np
import theano
import theano.tensor as T
from theano.tensor.signal import pool
import random

#import input data
neurons = scipy.io.loadmat('input_neuron')
neuron_input = neurons['WB']
neuron_train = neuron_input[0:80]
neuron_test = neuron_input[80:100]

force = scipy.io.loadmat('true.mat')
data_label=np.asarray(force['skt']).reshape(-1)
label_y=(data_label-np.mean(data_label))/np.std(data_label)
label_train=label_y[0:80]
label_test=label_y[80:100]

In [34]:
label_test.shape

(20L,)

In [45]:
def load_data(data1, data2):
    def shared_dataset(data1,data2,borrow=True):
        shared_x = theano.shared(np.asarray(data1, dtype=theano.config.floatX),borrow=True)
        shared_y = theano.shared(np.asarray(data2, dtype=theano.config.floatX),borrow=True)
        return shared_x, T.cast(shared_y,'float64')
    
    train_set_x, train_set_y = shared_dataset(data1,data2)
    rval = [(train_set_x, train_set_y)]
    return rval

class RBM_NN(object):
    def __init__(self, input, n_in, n_out):
        self.W = theano.shared(value=np.zeros((n_in, n_out),dtype=theano.config.floatX),name='W',borrow=True)
        self.b = theano.shared(value=np.zeros((n_out,),dtype=theano.config.floatX),name='b',borrow=True)               
        self.y_pred = T.dot(input, self.W)+self.b      
        self.params = [self.W, self.b]
        
    def difference(self,y):
        return T.mean(abs(self.y_pred-T.reshape(y,[10,1])))
    
class HiddenLayer(object):
    def __init__(self, rng, input, n_in, n_out):
        self.input = input
        W_values = np.asarray(rng.uniform(low=-np.sqrt(6./(n_in+n_out)),
                                          high=np.sqrt(6./(n_in+n_out)),
                                          size=(n_in,n_out)),
                              dtype=theano.config.floatX)
        W = theano.shared(value=W_values, name='W',borrow=True)
        
        b_values = np.zeros((n_out,),dtype=theano.config.floatX)
        b = theano.shared(value=b_values, name='b',borrow=True)
        
        self.W=W
        self.b=b
        
        lin_output = T.dot(input,self.W) + self.b
        self.output = T.nnet.sigmoid(lin_output)
        
        self.params = [self.W, self.b]

class MLP(object):
    def __init__(self, rng,input,n_in, n_hidden, n_out):
        self.hiddenLayer1 = HiddenLayer(rng,
                                       input=input,
                                       n_in=n_in,
                                       n_out=n_hidden)
        
        self.LR=RBM_NN(input=self.hiddenLayer1.output,
                      n_in=n_hidden,
                      n_out=n_out)
        
        self.L2_sqr = ((self.hiddenLayer1.W**2).sum()+
                       (self.LR.W**2).sum()
                      )
        
        
        self.error = (self.LR.difference)
        
        self.params = self.hiddenLayer1.params + self.LR.params
        
        self.input = input

        
def test_mlp(learning_rate=0.01, n_epochs=1000,  data1=neuron_train, data2=label_train,
             data3=neuron_test, data4=label_test, batch_size=10, n_hidden=1000):
    datasets_train = load_data(data1,data2)
    datasets_test = load_data(data3,data4)
    
    train_set_x, train_set_y = datasets_train[0]
    test_set_x, test_set_y = datasets_test[0]
    
    n_train_batches = train_set_x.get_value(borrow=True).shape[0] // batch_size
    n_test_batches = test_set_x.get_value(borrow=True).shape[0]//batch_size
    
    ######################
    # BUILD ACTUAL MODEL #
    ######################
    print('... building the model')
    index = T.lscalar()
    x = T.matrix('x')
    y = T.vector('y')    
    
    rng = np.random.RandomState(1234)
    
    classifier = MLP(rng=rng,
                    input=x,
                    n_in=200*200,
                    n_hidden=n_hidden,
                    n_out=1)
    
    cost=(classifier.error(y)+0.0001*classifier.L2_sqr)
    
    test_model = theano.function(inputs=[index],
                                outputs=classifier.error(y),
                                givens={
            x:test_set_x[index*batch_size:(index+1)*batch_size],
            y:test_set_y[index*batch_size:(index+1)*batch_size]
        })
    
    gparams = [T.grad(cost,param) for param in classifier.params]
    
    updates = [(param, param - learning_rate*gparam) for 
              param, gparam in zip(classifier.params, gparams)]
    
    train_model = theano.function(inputs=[index],
                                 outputs=cost,
                                 updates=updates,
                                 givens={
            x: train_set_x[index*batch_size:(index+1)*batch_size],
            y: train_set_y[index*batch_size:(index+1)*batch_size]
        })

    ###############
    # TRAIN MODEL #
    ###############
    print('... training the model')
    
    epoch=0 
    while (epoch<n_epochs):
        epoch = epoch + 1
        #print epoch
        #if epoch != 1:
        #    W=test_mlp2()
            
        for minibatch_index in range(n_train_batches): 
            minibatch_avg_cost=train_model(minibatch_index)    
            # iteration number
            iter = (epoch - 1) * n_train_batches + minibatch_index
            #print iter
            if (iter + 1) % 8 == 0:
                # compute zero-one loss on validation set
                losses = [train_model(i) for i in range(n_train_batches)]
                this_loss = np.mean(losses)
                
                print "Epoch {0}, Minibatch {1}/{2}, Test Error= {3}".format(epoch,minibatch_index+1,n_train_batches,
                                                                       this_loss)

        if epoch%10==0:
            test_losses = [test_model(i) for i in range(n_test_batches)]
            print test_losses
            test_score = np.mean(test_losses)
            
            print "Testing Score= {0}".format(test_score)
            
            
    return classifier.params
    

In [46]:
W=test_mlp()

... building the model
... training the model
Epoch 1, Minibatch 8/8, Test Error= 0.943471973512
Epoch 2, Minibatch 8/8, Test Error= 0.968223521329
Epoch 3, Minibatch 8/8, Test Error= 0.907507312838
Epoch 4, Minibatch 8/8, Test Error= 0.930128434004
Epoch 5, Minibatch 8/8, Test Error= 0.87718740184
Epoch 6, Minibatch 8/8, Test Error= 0.847316133485
Epoch 7, Minibatch 8/8, Test Error= 0.82345338587
Epoch 8, Minibatch 8/8, Test Error= 0.932339532924
Epoch 9, Minibatch 8/8, Test Error= 0.830162665901
Epoch 10, Minibatch 8/8, Test Error= 0.838289103166
[array(0.8436274721366901), array(1.1590887305512894)]
Testing Score= 1.00135810134
Epoch 11, Minibatch 8/8, Test Error= 0.756657173267
Epoch 12, Minibatch 8/8, Test Error= 0.792891459145
Epoch 13, Minibatch 8/8, Test Error= 0.707948360249
Epoch 14, Minibatch 8/8, Test Error= 0.659981064603
Epoch 15, Minibatch 8/8, Test Error= 0.688541540521
Epoch 16, Minibatch 8/8, Test Error= 0.66237269433
Epoch 17, Minibatch 8/8, Test Error= 0.71425049643